<a href="https://colab.research.google.com/github/atharvare9/Three_Gorges_Dam/blob/main/Three_gorges_reservoir_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
def june10_sept10(I, S):
    S_limit = storage['S145']
    if I + S > S_limit:
        Q = min(35000 * s_to_d, S + I - S_limit)
        Q = max(Q, 8000 * s_to_d)
    else:
        if I > 10000 * s_to_d:
            Q = min(35000 * s_to_d, I)
        else:
            Q = 8000 * s_to_d
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q
# not completed

In [ ]:
def sept11_sept31(I, S, day):
    S_limit = S + (storage['S162'] - S) / (22 - day)
    if I + S > S_limit:
        Q = min(35000 * s_to_d, S + I - S_limit)
        Q = max(Q, 8000 * s_to_d) 
    else:
        if I > 10000 * s_to_d:
            Q = min(35000 * s_to_d, I)
        else:
            Q = 8000 * s_to_d
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q

In [ ]:
def oct1_oct31(I, S, day):
    S_limit = S + (storage['S175'] - S) / (32 - day)
    if I + S > S_limit:
        Q = min(35000 * s_to_d, S + I - S_limit)
        Q = max(Q, 8000 * s_to_d)
    else:
        if I > 8000 * s_to_d:
            Q = min(35000 * s_to_d, I)
        else:
            Q = 8000 * s_to_d
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q

In [ ]:
def nov1_dec31(I, S):
    S_limit = storage['S175']
    Q = max(5300 * s_to_d, I + S - S_limit)
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q

In [ ]:
def jan1_april31(I, S, day):
    S_limit = S - (S - storage['S155']) / (121 - day)
    Q = min(35000 * s_to_d, S + I - S_limit)
    Q = max(6000 * s_to_d, Q)
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q

In [ ]:
def may1_may25(I, S, day):
    if S > storage['S155']:
        S_limit = S - (S - storage['S155']) / (26 - day)
        Q = min(35000 * s_to_d, S + I - S_limit)
        Q = max(Q, 6000 * s_to_d)
    else:
        S_limit = storage['S155']
        Q = min(35000 * s_to_d, S + I - S_limit)
        Q = max(6000 * s_to_d, Q)
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
      if S1 < 17:
        Q = max(I + S - 17, 0)
    return Q

In [ ]:
def may26_june10(I, S, day):
    S_limit = S - (S - storage['S145']) / (17 - day)
    Q = min(35000 * s_to_d, S + I - S_limit)
    Q = max(6000 * s_to_d, Q)
    S1 = I + S - Q
    if S1 > storage['S175']:
      Q =max(35000 * s_to_d, S + I - storage['S175'])
    if S1 < 17:
      Q = max(I + S - 17, 0)
    return Q

In [ ]:
def storage_to_elev(S):
    A = 96692.07
    B = 2310
    return ((-A*B + (A**2 * B**2 + 12 * B * S * 10**9 )**0.5)/ (6*B))

In [ ]:
def calc_hydro(S, Q):
    up_elev = storage_to_elev(S)
    down_elev = 62
    H = up_elev - down_elev
    E = 0.92
    power = E * 9810 * Q * H  / 86400
    for t in range(len(power)):
      if power[t] > 22.5:
        power[t] = 22.5
    return power, up_elev

In [ ]:
def calc_storage(data, Inflow): 
    S1 = np.zeros(len(Inflow))
    Q1 = np.zeros(len(Inflow))
    S1[0] = 39.3
    Q1[0] = Inflow[0]
    for t in range(len(Inflow) - 1):
        S1[t + 1] = S1[t] + Inflow[t] - Q1[t]
        if ((data['month'].iloc[t] == 6 and data['day'].iloc[t] >10) or (data['month'].iloc[t] == 7)
            or (data['month'].iloc[t] == 8) or (data['month'].iloc[t] == 9 and data['day'].iloc[t] <=10)):
            Q1[t + 1] = june10_sept10(Inflow[t + 1], S1[t + 1])

        elif (data['month'].iloc[t] == 9 and (data['day'].iloc[t] >10 and data['day'].iloc[t] <= 31)):
            if data['month'].iloc[t] == 9:
                day = data['day'].iloc[t] - 10
            if data['month'].iloc[t] == 10:
                day = 21 + data['day'].iloc[t]
            Q1[t + 1] = sept11_sept31(Inflow[t + 1], S1[t + 1], day)

        elif data['month'][t] == 10:
            day = data['day'].iloc[t]
            Q1[t + 1] = oct1_oct31(Inflow[t + 1], S1[t + 1], day)

        elif data['month'].iloc[t] == 11 or data['month'].iloc[t] == 12:
            Q1[t + 1] = nov1_dec31(Inflow[t + 1], S1[t + 1])

        elif (data['month'].iloc[t] == 1 or data['month'].iloc[t] == 2 or data['month'].iloc[t] == 3 or 
            data['month'].iloc[t] == 4):
            if data['month'][t] == 1:
                day = data['day'].iloc[t]
            elif data['month'][t] == 2:
                day = 31 + data['day'].iloc[t]
            elif data['month'][t] == 3:
                day = 31 + 28 + data['day'].iloc[t]
            else:
                day = 31 + 28 + 31 + data['day'].iloc[t]
            Q1[t + 1] = jan1_april31(Inflow[t + 1], S1[t + 1], day)

        elif data['month'].iloc[t] == 5 and data['day'][t] <= 25:
            day = data['day'].iloc[t]
            Q1[t + 1] = may1_may25(Inflow[t + 1], S1[t + 1], day)

        else:
            if data['month'].iloc[t] == 5:
                day = data['day'].iloc[t] - 25
            else:
                day = data['day'].iloc[t] + 6
            Q1[t + 1] = may26_june10(Inflow[t + 1], S1[t + 1], day)
    return S1, Q1


In [ ]:

data = pd.read_csv("gdrive/My Drive/Colab Notebooks/Three gorges project/Version 3 Data.csv",header = None, names = ['date','day','month', 'year', 'inflow'])
s_to_d = 86400 * 10 **(-9) 
storage = {'S145': 32.53, 'S162': 36.37, 'S175': 39.3, 'S155': 34.79 , 'S181': 40.65 }
df = pd.DataFrame(data['inflow'])
Inflow = df.to_numpy()
Inflow = Inflow[:,0]
Inflow = Inflow * 86400 * 10**(-9)
S, outflow = calc_storage(data, Inflow)
plt.figure(figsize = (20, 10))
plt.plot(Inflow, linewidth = 0.5, label = 'Inflow')
plt.plot(S, linewidth = 0.5, label = 'Storage')
plt.plot(outflow, linewidth = 0.5, label = 'Outflow')
np.savetxt('gdrive/My Drive/Colab Notebooks/Three gorges project/basenew.csv', outflow, delimiter = ',')
np.savetxt('gdrive/My Drive/Colab Notebooks/Three gorges project/initial_storagenew.csv', S, delimiter = ',')
plt.legend()
plt.savefig('trial1curve.png', dpi = 300)
plt.show()
hydropower, upelev = calc_hydro(S, outflow)
np.savetxt('gdrive/My Drive/Colab Notebooks/Three gorges project/hydro_initialnew.csv', hydropower, delimiter = ',')
np.savetxt('gdrive/My Drive/Colab Notebooks/Three gorges project/upelevnew.csv', upelev, delimiter = ',')

